In [ ]:
import threading
from pathlib import Path

import lib_ipmu_daq as daq
import lib_ipmu_daq_config as config
import lib_ipmu_daq_generator as gen
import lib_ipmu_daq_process as post
import lib_ipmu_daq_graph as graph

%load_ext autoreload
%autoreload 2

Exception in thread Thread-115 (run):
Traceback (most recent call last):
  File "c:\Users\USER\Desktop\ipmu_DAQ\src\lib_ipmu_daq_process.py", line 73, in _processorLoop
Exception in thread Thread-101 (run):
Traceback (most recent call last):
  File "c:\Users\USER\Desktop\ipmu_DAQ\src\lib_ipmu_daq_process.py", line 139, in _processorLoop
    ring_Iu.extend(Iu); ring_Iv.extend(Iv); ring_Iw.extend(Iw)
                                        ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\queue.py", line 199, in get_nowait
    return np.sum(dir_log)
                           
  File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\queue.py", line 199, in get_nowait
    return self.get(block=False)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\queue.py", line 168, in get
    return self.get(block=False)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\AppData\Local\Programs\Python

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
preset = config._loadConfig(Path("./_config_preset.toml"))
run = config._loadConfig(Path("./_config_run.toml"))
app_config = config.initParams(preset, run, debug=True)


[Io]
  sample_rate         : 100000
  gen_chunk_sec       : 0.100000
  proc_interval       : 0.125000
  queue_depth         : 40
  quad_depth          : 40

[Gui]
  display_sec         : 3600
  plot_sec            : 0.150000
  gui_interval_ms     : 50
  pruning             : 100

[Logging]
  log_chunk           : 1024
  log_data_num        : 4

[Encoder Postproc]
  quadpulse_width     : 0.000250
  threshold           : 2.500000

[Debug Encoder]
  input_velocity      : 0.100000
  pulse_height        : 5.000000
  pulse_duty          : 0.500000
  pulse_phase_A       : 0.000000

[Debug Power]
  amplitude           : 0.300000
  phase               : 0.000000

[Driver]
  t_DC                : 10
  pps                 : 0
  step                : 100
  rst                 : 1
  target_freq         : 1

[Dependent]
  chunk_sec           : 0.100000
  n_samples_gen       : 10000
  samples_proc        : 12500
  history             : 150
  count_history       : 80
  velo_history        : 80
  pow_

In [63]:
app = daq.DAQApp(config=app_config)
app.setup() # initialize widgets

generator = gen.Generator(config=app_config, buf_q=app.buf_q, stop_event=app.stop_event)
processor = post.Processor(config=app_config, buf_q=app.buf_q, quad_q=app.quad_q, stop_event=app.stop_event, runs_dir=config.RUNS_DIR)
gui = graph.DAQGUI(config=app_config, quad_q=app.quad_q, stop_event=app.stop_event)

# 5. Create the thread objects individually
generator_thread = threading.Thread(target=generator.run, daemon=True)
processor_thread = threading.Thread(target=processor.run, daemon=True)

# 6. Start the threads (now controlled from the top level)
print("Starting threads from main...")
generator_thread.start()
processor_thread.start()

gui.run()

generator_thread.join(timeout=0.1)
processor_thread.join(timeout=0.1)



Shared resources (Queues, Stop Event) are set up.
Starting threads from main...
HDF5 dataset created at: C:\Users\USER\Desktop\ipmu_DAQ\src\..\runs\251003203733.h5
